In [0]:
pip install deap

     |████████████████████████████████| 153kB 2.2MB/s 


In [30]:
#    example which maximizes the sum of a list of integers
#    each of which can be 0 or 1

import random

from deap import base
from deap import creator
from deap import tools

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Attribute generator 
#                      define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,1] (i.e. 0 or 1 with equal
#                      probability)
# 가질 수 있는 범위 : 여기서는 0, 1, 2 셋 중 하나
toolbox.register("attr_bool", random.randint, 0, 2)

# Structure initializers
#                         define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
# 이 부분에서 길이를 지정
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 100)

# define the population to be a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# the goal ('fitness') function to be maximized
# evaluation function : 여기서는 그냥 리스트 원소의 합
def evalOneMax(individual):
    return sum(individual),

#----------
# Operator registration
#----------
# register the goal / fitness function
# 위에서 정의한 함수 여기에 적어주기.
toolbox.register("evaluate", evalOneMax)

# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
toolbox.register("select", tools.selTournament, tournsize=3)

#----------

def main():
    random.seed(64)

    # create an initial population of 300 individuals (where
    # each individual is a list of integers)
    # 처음에 300개 생성
    # 300개 길이의 리스트, 각 원소는 100개(지정한 길이) 길이의 리스트
    # len(pop) -> 300 출력됨.
    # len(pop[0]) -> 100 출력됨.
    pop = toolbox.population(n=300)

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.2
    
    print("Start of evolution")
    
    # Evaluate the entire population
    # print(fitnesses) -> [(84,), (105,), (96,), (104,), (94,),  ... ] 이런식으로 저장됨.
    # i 번째 pop의 fitness 접근은 fitnesses[i][0] 이런식으로 이용하면 됨.
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    
    # pop 개수만큼 evaluated 됨.
    print("  Evaluated %i individuals" % len(pop))

    # Extracting all the fitnesses of 
    # print(fits) -> [84.0, 105.0, 96.0, 104.0, 94.0, ... ] 이런 식으로 저장됨.
    # 위에 fitnesses 를 다듬는 작업!
    fits = [ind.fitness.values[0] for ind in pop]
    # Variable keeping track of the number of generations
    g = 0
    
    # Begin the evolution
    while max(fits) < 180 and g < 300:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        # 300개 길이의 리스트, 각 원소는 100개(지정한 길이) 길이의 리스트
        # len(pop) -> 300 출력됨.
        # len(pop[0]) -> 100 출력됨.
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
    
        # Apply crossover and mutation on the offspring
        # offspring[::2] 하면 0번 원소부터 2칸씩 -> 총 150개
        # offspring[1::2] 하면 1번 원소부터 2칸씩 -> 총 150개
        # 예를 들어서, a = [1, 2, 3, 4, 5, 6] 일때,
        # a[::2] -> [1, 3, 5]
        # a[1::2] -> [2, 4, 6]
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        print("  Evaluated %i individuals" % len(invalid_ind))
        
        # The population is entirely replaced by the offspring
        pop[:] = offspring
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)


    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    print(best_ind.fitness.values)

main()

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Start of evolution
  Evaluated 300 individuals
-- Generation 1 --
  Evaluated 203 individuals
  Min 90.0
  Max 123.0
  Avg 107.37666666666667
  Std 5.9375743270202
-- Generation 2 --
  Evaluated 171 individuals
  Min 95.0
  Max 126.0
  Avg 111.93666666666667
  Std 5.337851211447894
-- Generation 3 --
  Evaluated 190 individuals
  Min 98.0
  Max 136.0
  Avg 115.63
  Std 5.516620342202297
-- Generation 4 --
  Evaluated 201 individuals
  Min 101.0
  Max 137.0
  Avg 119.26333333333334
  Std 5.903726694968876
-- Generation 5 --
  Evaluated 175 individuals
  Min 108.0
  Max 139.0
  Avg 123.33333333333333
  Std 6.251044357190005
-- Generation 6 --
  Evaluated 165 individuals
  Min 112.0
  Max 143.0
  Avg 127.93
  Std 5.59509606709284
-- Generation 7 --
  Evaluated 169 individuals
  Min 116.0
  Max 144.0
  Avg 131.28666666666666
  Std 4.999115477317508
-- Generation 8 --
  Evaluated 192 individuals
  Min 115.0
  Max 148.0
  Avg 134.34666666666666
  Std 5.14261498548034
-- Generation 9 --
  Eva

In [0]:
# evaluation function : 리스트의 합
# 리스트 길이 : 100
# population : 300 개
# 각 원소는 0, 1, 2 셋 중 하나로 생성될 수 있음.
# 만약 generation 이 300을 넘어가거나 / fitness 가 180을 넘으면 stop.
# 최종 fitness 는 183